In [96]:
import ast
from pathlib import Path

from ipymarkup import show_span_line_markup, show_span_box_markup
import pandas as pd

In [326]:
dffile = Path("../../data/ner/baseline/train.xlsx")

if not dffile.exists():
    print(f"No file at {dffile}")

# Load Data

In [338]:
df = pd.read_excel(dffile, engine="openpyxl")
df.dropna(subset=["markup", "markup_classicNER"], inplace=True)
df["markup"] = df["markup"].apply(ast.literal_eval)
df["markup_classicNER"] = df["markup_classicNER"].apply(ast.literal_eval)
df["glued_markup"] = df["glued_markup"].apply(ast.literal_eval)
print(df.shape)
df.head()

(814, 7)


In [328]:
def display_n_docs(df, n=10, random_state=777):
    """
    Helper function to visualize the markup with bboxes
    """
    
    _df = df.sample(
        n=n,     
        random_state=random_state
    )
    
    for i, row in _df.iterrows():
        
        doc_name, text, spans = row["doc_name"], row["text"], row["markup"]
        
        print(f"Showing {doc_name}")
        show_span_line_markup(text, spans) 

In [337]:
# DOCS_TO_DISPLAY = 10

# display_n_docs(
#     df=df,
#     n=DOCS_TO_DISPLAY,
# )

# Fix Markup (Skip If alreadyd did it)

In [312]:
# Join Overlapping Entities to work with Augmenty

def heuristic_filter_zeros(entities, text):
    
    entities_filtered = []
    
    for entity  in entities:
        
        start, end, label = entity[0], entity[1], entity[2]
        value = text[start:end]
        
        if value == "0,00":
            continue
                        
        entities_filtered.append(entity)
    
    return entities_filtered


def merge_close_entities(entities, text):
    
    entities_merged = []
    entities_merged.append(entities[0])
    
    entities.sort(key= lambda x: (x[2], x[0]))
    
    for i in range(1, len(entities)):
        
        start, end, label = entities[i][0], entities[i][1], entities[i][2]
        start_prev_entity, end_prev_entity, label_prev_entity = entities[i-1][0], entities[i-1][1], entities[i-1][2]
            
        if start == end_prev_entity+1 and label == label_prev_entity:
            
            if "," in text[start:end] and "," in text[start_prev_entity:end_prev_entity]:
                entities_merged.append(entities[i])
            
            else:
                entity_merged = (start_prev_entity, end, label)
                entities_merged.pop(-1)
                entities_merged.append(entity_merged)
            
        else:
            
            entities_merged.append(entities[i])    
    
    return entities_merged


def join_overlap_entities(entities, text):

    def _calculate_overlap(start, end, initial_range):
        
        entity_range = set(list(range(start, end+1)))
        initial_range = set(initial_range)
        
        overlap = len(entity_range & initial_range) / len(entity_range | initial_range)
#         print(f"{entity_range}_{entity_range}: {overlap}")
        
        return overlap
    
    entities_joined = []
    entities.sort(key=lambda x: (x[0], x[2]))
    
    for i in range(len(entities)):
        
        if i > len(entities) -1:
            break
    
        start, end, label = entities[i][0], entities[i][1], entities[i][2]
        entity_range = list(range(start, end+1))
        
        entities_with_overlap = [entity for entity in entities[i:] if _calculate_overlap(
            entity[0],
            entity[1],
            entity_range)
                                >=0.7]
        
        if len(entities_with_overlap) == 1:
            entities_joined.append(entities[i])
            continue
        
#         print(f"Found {len(entities_with_overlap)}")
        
        starts = [entity[0] for entity in entities_with_overlap]
        ends = [entity[1] for entity in entities_with_overlap]
        labels = [entity[2] for entity in entities_with_overlap]
        
        entity_joined_start = min(starts)
        entity_joined_end = max(ends)
        entity_joined_label = "".join(sorted(labels))        
        
        entities_joined.append((entity_joined_start, entity_joined_end, entity_joined_label))
        
        for entity_relevant in entities_with_overlap[1:]:
            if entity_relevant in entities:
                entities.remove(entity_relevant)
                    
    return entities_joined


def fix_markup(markup, text, do_join=False):
    
    if markup is None:
        return None
    
    markup = heuristic_filter_zeros(markup, text)
    
    if len(markup) == 0:
        return None
    
    markup = merge_close_entities(markup, text)
    
    if do_join:
        markup = join_overlap_entities(markup, text)
    
    return markup

In [339]:
# Gather Dataset Again 

df["markup"] = df.apply(lambda x: fix_markup(x.markup, x.text), axis=1)
df["markup_classicNER"] = df.apply(lambda x: fix_markup(x.markup, x.text, do_join=True), axis=1)

df.head()

In [315]:
df.to_excel(dffile, index=False, encoding="utf-8")

# EDA Entities

In [330]:
# MultiLabel

all_entities = []


markup = df["markup"].tolist()
for m in markup:
    labels = [entity[2] for entity in m]
    all_entities.extend(labels)
    
entities = pd.Series(all_entities)
entities.value_counts()

ИтоговаяСумма    2822
ПоДеталям        1039
ПоРаботам        1009
Скидки            441
СоСкидкой         407
СНДС               96
dtype: int64

In [333]:
# Multiclass

all_entities = []


markup = df["markup_classicNER"].tolist()
for m in markup:
    labels = [entity[2] for entity in m]
    all_entities.extend(labels)
    
entities = pd.Series(all_entities)
entities.value_counts()

ИтоговаяСумма                                   904
ИтоговаяСуммаПоРаботам                          747
ИтоговаяСуммаПоДеталям                          519
Скидки                                          232
ИтоговаяСуммаПоДеталямСоСкидкой                 221
ИтоговаяСуммаПоДеталямСкидки                     96
ПоРаботам                                        82
ПоДеталям                                        77
ИтоговаяСуммаСоСкидкой                           60
ИтоговаяСуммаСкидки                              59
ИтоговаяСуммаПоРаботамСоСкидкой                  52
СоСкидкой                                        40
ИтоговаяСуммаПоДеталямПоРаботам                  35
ИтоговаяСуммаСНДС                                24
ИтоговаяСуммаПоРаботамСНДС                       24
ПоДеталямСкидки                                  22
ИтоговаяСуммаПоДеталямСНДС                       19
ПоРаботамПоРаботамПоРаботам                      14
ИтоговаяСуммаПоДеталямСНДССоСкидкой              11
ПоДеталямСоС